In [39]:
from f2b.f2b_estimation.data import Data
from f2b.f2b_estimation.likelihood import Likelihood
from f2b.f2b_estimation.maximum_likelihood_estimation import LineSearch
from f2b.postprocessing.output_analysis import get_estimation_results_and_indicators

In [40]:
from numpy import array
from datetime import timedelta

In [41]:
import os
os.chdir('/home/justine/Nextcloud/Cired/Recherche/Econometrie/fail_to_board_probability/f2b_code')

In [42]:
origin_station = "VIN"
destination_stations = ["NAT", "LYO", "CHL", "AUB", "ETO", "DEF"]
date = "04/02/2020"
direction = 2

data = Data(date, origin_station, destination_stations, direction)
f2b_probas = array([0.0 for _ in range(data.runs_number)])

Trips construction...


100%|██████████| 8336/8336 [00:01<00:00, 4822.89it/s]


Runs construction...


100%|██████████| 308/308 [00:00<00:00, 2431.06it/s]


##### Shift all run's departure and arrival times at all stations.

In [43]:
for run_code in data.runs_chronological_order:
    for departure_station_code in data.runs[run_code].departure_times:
        data.runs[run_code].departure_times[departure_station_code] += timedelta(seconds = 10)
    for arrival_station_code in data.runs[run_code].arrival_times:
        data.runs[run_code].arrival_times[arrival_station_code] += timedelta(seconds = 10)

##### Change runs <-> trips associations accordingly.

In [44]:
for run_code in data.runs_chronological_order:
    data.runs[run_code].associated_trips = []
    data.runs[run_code].associated_trips_number = 0

for trip_id in data.trips:
    data.trips[trip_id].associated_runs = []
    data.trips[trip_id].associated_runs_number = 0
    origin_station = data.trips[trip_id].access_station
    destination_station = data.trips[trip_id].egress_station
    for run_code in data.runs_chronological_order:
        if data.runs[run_code].departure_times[origin_station] > data.trips[trip_id].access_time:
            if destination_station in data.runs[run_code].arrival_times:
                if data.runs[run_code].arrival_times[destination_station] < data.trips[trip_id].egress_time:
                    data.trips[trip_id].associated_runs.append(run_code)
                    data.trips[trip_id].associated_runs_number +=1

                    data.runs[run_code].associated_trips.append(trip_id)
                    data.runs[run_code].associated_trips_number += 1
       


In [45]:
matching_error1 = 0
for trip_id in data.trips:
    for run_code in data.trips[trip_id].associated_runs:
        if trip_id not in data.runs[run_code].associated_trips:
            matching_error1 += 1

matching_error2 = 0
for run_code in data.runs_chronological_order:
    for trip_id in data.runs[run_code].associated_trips:
        if run_code not in data.trips[trip_id].associated_runs:
            matching_error2 += 1
print(matching_error1)
print(matching_error2)

0
0


In [50]:
data.trips[593250].associated_runs
print(data.runs['YSTO57'].departure_times['VIN'])
print(data.runs['YSTO57'].arrival_times['LYO'])

2020-04-02 18:58:01
2020-04-02 19:03:22


In [51]:
likelihood = Likelihood(data, f2b_probas)

(Offline) Compute access and egress individual likelihood contributions...


100%|██████████| 8336/8336 [00:03<00:00, 2421.84it/s]


ValueError: math domain error

In [47]:
MLE = LineSearch(
    likelihood=likelihood,
    initial_f2b_probas=f2b_probas,
    global_tolerance=0.1,
    max_iteration=8,
    local_tolerance=0.01,
    max_local_binary_iteration=20,
)
MLE.run_iterative_optimization()
print(MLE.log_likelihood_by_iteration)


(Offline) Compute access and egress individual likelihood contributions...


100%|██████████| 8336/8336 [00:03<00:00, 2614.57it/s]


ValueError: math domain error

In [ ]:
estimation_results = get_estimation_results_and_indicators(data, likelihood)
estimation_results_non_zero_proba = estimation_results[
    estimation_results["estimated_f2b"] != 0
].copy()
print(estimation_results_non_zero_proba["asymptotic_confidence"].describe())
estimation_results.to_csv(f"output/estimation_info_plus_10s_{origin_station}.csv")

count    143.000000
mean       0.000268
std        0.001144
min       -0.001348
25%        0.000019
50%        0.000055
75%        0.000150
max        0.011979
Name: asymptotic_confidence, dtype: float64
